In [1]:
# Program places orders

# STATUS: Incomplete

#***          Start ib_insync (run once)       *****
#___________________________________________________

from ib_insync import *
util.startLoop()
ib=IB().connect('127.0.0.1', 7496, clientId=1) # kavi TWS live
# ib=IB().connect('127.0.0.1', 4001, clientId=1) # kavi IBG live

In [ ]:
#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'
import pandas as pd

# Read the put thresholds
df_th = pd.read_csv(datapath+'put_threshold1.csv', sep=',', 
            header=None).rename(columns={1: 'ibSymbol', 4: 'Threshold'}).drop([0,2,3],1)

df_th.Threshold = df_th.Threshold.astype(float)

# Read the preorder puts
df_puts = pd.read_pickle(datapath+'df_preorder_puts.pkl')

# Filter based on thresholds
cols = ['ibSymbol', 'undPrice', 'Strike', 'Expiry', 'DTE', 
        'pLTP', 'pBidPrice', 'tgt_price', 'Type', 'Mlot', 'Delta', 'RoM']
df_puts1 = df_puts[cols]
df_puts2 = df_puts1.merge(df_th)
df_puts3 = df_puts2[df_puts2.Strike <= df_puts2.Threshold].reset_index(drop=True)

# convert expiry to the format required by IBKR
df_puts3.Expiry = df_puts3.Expiry.dt.strftime('%Y%m%d')

#****       Placing the Orders    ********

# Qualify the contracts
c_list = [Option(symbol=symbol, exchange='NSE', 
       lastTradeDateOrContractMonth=expiry, strike=str(strike), right='P') 
for symbol, strike, expiry in zip(df_puts3.ibSymbol, df_puts3.Strike, df_puts3.Expiry)]

contracts = [c for i in range(0, len(c_list), 50) for c in ib.qualifyContracts(*c_list[i: i+50])]

df_puts3['Contracts'] = contracts

In [93]:
cols = ['ibSymbol', 'Expiry', 'Strike', 'Type']
df_contracts = pd.DataFrame([(c.symbol, c.lastTradeDateOrContractMonth, c.strike, c.right) 
              for c in contracts])

df_contracts.columns=cols

In [116]:
df_contracts

,ibSymbol,Expiry,Strike,Type
0,KOTAKBANK,20181227,940.0,P
1,KOTAKBANK,20181227,900.0,P
2,KOTAKBANK,20181227,920.0,P
3,KOTAKBANK,20181227,880.0,P
4,COLPAL,20181227,880.0,P
5,COLPAL,20181227,900.0,P
6,COLPAL,20181227,920.0,P
7,COLPAL,20181227,940.0,P
8,LUPIN,20181227,720.0,P
9,GRASIM,20181227,640.0,P


In [117]:
pd.merge(df_contracts.set_index(cols), df_puts3.set_index(cols), left_index=True, right_index=True)
df_contracts.set_index(cols).join(df_puts3.set_index(cols))

undPrice  DTE  pLTP  pBidPrice  tgt_price  \
ibSymbol  Expiry   Strike Type                                              
KOTAKBANK 20181227 940.0  P          NaN  NaN   NaN        NaN        NaN   
                   900.0  P          NaN  NaN   NaN        NaN        NaN   
                   920.0  P          NaN  NaN   NaN        NaN        NaN   
                   880.0  P          NaN  NaN   NaN        NaN        NaN   
COLPAL    20181227 880.0  P          NaN  NaN   NaN        NaN        NaN   
                   900.0  P          NaN  NaN   NaN        NaN        NaN   
                   920.0  P          NaN  NaN   NaN        NaN        NaN   
                   940.0  P          NaN  NaN   NaN        NaN        NaN   
LUPIN     20181227 720.0  P          NaN  NaN   NaN        NaN        NaN   
GRASIM    20181227 640.0  P          NaN  NaN   NaN        NaN        NaN   
                   740.0  P          NaN  NaN   NaN        NaN        NaN   
                   720.0  P          NaN  NaN   NaN        NaN        NaN   
                   660.0  P          NaN  NaN   NaN        NaN        NaN   
                   680.0  P          NaN  NaN   NaN        NaN        NaN   
                   700.0  P          NaN  NaN   NaN        NaN        NaN   
                   620.0  P          NaN  NaN   NaN        NaN        NaN   
KSCL      20181227 450.0  P          NaN  NaN   NaN        NaN        NaN   
                   440.0  P          NaN  NaN   NaN        NaN        NaN   
                   420.0  P          NaN  NaN   NaN        NaN        NaN   
                   430.0  P          NaN  NaN   NaN        NaN        NaN   
                   410.0  P          NaN  NaN   NaN        NaN        NaN   
                   400.0  P          NaN  NaN   NaN        NaN        NaN   
IBULHSGFI 20181227 600.0  P          NaN  NaN   NaN        NaN        NaN   
                   560.0  P          NaN  NaN   NaN        NaN        NaN   
                   580.0  P          NaN  NaN   NaN        NaN        NaN   
                   540.0  P          NaN  NaN   NaN        NaN        NaN   
                   520.0  P          NaN  NaN   NaN        NaN        NaN   
                   500.0  P          NaN  NaN   NaN        NaN        NaN   
CEATLTD   20181227 940.0  P          NaN  NaN   NaN        NaN        NaN   
                   960.0  P          NaN  NaN   NaN        NaN        NaN   
...                                  ...  ...   ...        ...        ...   
INDUSINDB 20181227 1240.0 P          NaN  NaN   NaN        NaN        NaN   
                   1260.0 P          NaN  NaN   NaN        NaN        NaN   
                   1280.0 P          NaN  NaN   NaN        NaN        NaN   
                   1160.0 P          NaN  NaN   NaN        NaN        NaN   
                   1180.0 P          NaN  NaN   NaN        NaN        NaN   
JINDALSTE 20181227 110.0  P          NaN  NaN   NaN        NaN        NaN   
IRB       20181227 100.0  P          NaN  NaN   NaN        NaN        NaN   
                   105.0  P          NaN  NaN   NaN        NaN        NaN   
                   110.0  P          NaN  NaN   NaN        NaN        NaN   
BANKINDIA 20181227 70.0   P          NaN  NaN   NaN        NaN        NaN   
                   67.5   P          NaN  NaN   NaN        NaN        NaN   
BATAINDIA 20181227 680.0  P          NaN  NaN   NaN        NaN        NaN   
GSFC      20181227 75.0   P          NaN  NaN   NaN        NaN        NaN   
LICHSGFIN 20181227 330.0  P          NaN  NaN   NaN        NaN        NaN   
                   340.0  P          NaN  NaN   NaN        NaN        NaN   
                   300.0  P          NaN  NaN   NaN        NaN        NaN   
                   310.0  P          NaN  NaN   NaN        NaN        NaN   
                   320.0  P          NaN  NaN   NaN        NaN        NaN   
                   350.0  P          NaN  NaN   NaN        NaN        NaN   
                   360.0  P 

In [110]:
df_puts3.dtypes

ibSymbol      object
undPrice     float64
Strike       float64
Expiry        object
DTE          float64
pLTP         float64
pBidPrice    float64
tgt_price    float64
Type          object
Mlot         float64
Delta        float64
RoM          float64
Threshold    float64
dtype: object

In [111]:
df_contracts.dtypes

ibSymbol     object
Expiry       object
Strike      float64
Type         object
dtype: object

In [ ]:
df_puts3['Orders'] = [LimitOrder(action='SELL', totalQuantity=mlot, lmtPrice=price) 
for mlot, price in zip(df_puts3.Mlot, df_puts3.tgt_price)]

In [ ]:
limitTrade = [ib.placeOrder(contract, order) 
              for contract, order in zip(df_puts3.Contracts, df_puts3.Orders)]